### <b>Bidirectional Recurrent Neural Network (BiRNN)</b>
A Bidirectional Recurrent Neural Network (BiRNN) is an extension of the traditional RNN that improves its ability to capture long-range dependencies in sequential data. It achieves this by processing the input sequence in both forward and backward directions, effectively providing two sets of hidden states for each time step.

#### How It Works
##### A BiRNN consists of two RNNs:

* <b>Forward RNN</b> : Processes the sequence from the first to the last time step.
* <b>Backward RNN</b> : Processes the sequence from the last to the first time step.

At each time step 𝑡 , the output is a combination of the hidden states from both the forward and backward RNNs:

        ℎ𝑡=concatenate(ℎ𝑡 forward,ℎ𝑡 backward)

This structure allows the model to consider both past and future context when making predictions.

In [3]:
# A Bidirectional Simple RNN for Text Classification using the IMDB movie reviews dataset.

# Importing libraries
import tensorflow as tf

from tensorflow.keras.datasets import imdb 
# dataset contains 50,000 movie reviews labeled as positive or negative.

from tensorflow.keras.preprocessing.sequence import pad_sequences 
# ensures that all sequences have the same length.

from tensorflow.keras.models import Sequential # 

from tensorflow.keras.layers import Embedding, SimpleRNN, Bidirectional, Dense
# Embedding:  converts words into dense vectors.
# SimpleRNN:  is the core RNN layer.
# Bidirectional: makes the RNN process in both directions.
# Dense: is the output layer.

tf.random.set_seed(42)
# Set random seed for reproduciblity

In [4]:
# Load IMDB dataset (only keep the top 10,000 most frequent words)
vocab_size = 10000
max_length = 100  # Maximum review length

# We only keep the top 10,000 most common words to reduce model complexity.
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

# Pad sequences to ensure all reviews are of equal length
X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

print(f"Training data shape: {X_train.shape}, Labels shape: {y_train.shape}")
print(f"Testing data shape: {X_test.shape}, Labels shape: {y_test.shape}")

Training data shape: (25000, 100), Labels shape: (25000,)
Testing data shape: (25000, 100), Labels shape: (25000,)


In [8]:
# Defining the Bidirectional RNN model

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=32, input_length=max_length),
    Bidirectional(SimpleRNN(64, return_sequences=False)),
    Dense(1, activation='sigmoid')
])
# Embedding layer: Converts word indices into 32-dimensional vectors.

# Bidirectional SimpleRNN layer: 
#   Reads text forward and backward to capture both past and future context.
#   Uses 64 hidden units (each direction has 64, so total 128 parameters).
#   return_sequences=False: keeps only the last output for classification.

# Dense layer: Outputs a probability (binary classification).

/home/vpsr/.local/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [10]:
# Compile the Model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# adam: optimizes learning.
# binary_crossentropy: is the loss function for binary classification.
# accuracy: tracks model performance.

In [11]:
# Model summary
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [12]:
# Train the Model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))
# 5 epochs: Trains the model 5 times over the dataset.
# Batch size of 64: Processes 64 reviews at a time.
# Validation on test data: Measures generalization.

Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 26s 52ms/step - accuracy: 0.5462 - loss: 0.6815 - val_accuracy: 0.7356 - val_loss: 0.5395
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.6168 - loss: 0.6679 - val_accuracy: 0.7754 - val_loss: 0.4726
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.8096 - loss: 0.4292 - val_accuracy: 0.7608 - val_loss: 0.5033
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step - accuracy: 0.8629 - loss: 0.3330 - val_accuracy: 0.7704 - val_loss: 0.5333
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 52ms/step - accuracy: 0.9005 - loss: 0.2535 - val_accuracy: 0.7551 - val_loss: 0.6279


In [13]:
# Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
# Evaluates performance on unseen test data.

print(f"Test Accuracy: {accuracy:.4f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.7554 - loss: 0.6263
Test Accuracy: 0.7551


In [14]:
model.save("imdb_reviews_sentiment_analysis_using_bidirectional_simple_rnn.h5")

In [30]:
# Preparing a New Review for Prediction

# Since our model expects numeric input, we must tokenize and pad new reviews before passing them 
# into the model.

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define a sample review
new_reviews = ["This movie was absolutely fantastic! I loved it.",
               "Worst movie ever. I regret watching it."]

# Tokenizer with the same vocabulary size
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(new_reviews)  # Tokenize new text

# Convert text to sequences
new_sequences = tokenizer.texts_to_sequences(new_reviews)

# Pad sequences to match model input size (max_length=100)
new_padded = pad_sequences(new_sequences, maxlen=100, padding='post')


In [31]:
# Make predictions
predictions = model.predict(new_padded)

# Convert probabilities to labels (0 = negative, 1 = positive)
labels = ["Negative" if pred < 0.5 else "Positive" for pred in predictions]

# Print results
for review, sentiment, pred in zip(new_reviews, labels, predictions):
    print(f"Review: {review}\nPredicted Sentiment: {sentiment} with {pred*100}% accuracy\n")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Review: This movie was absolutely fantastic! I loved it.
Predicted Sentiment: Negative with [45.786903]% accuracy

Review: Worst movie ever. I regret watching it.
Predicted Sentiment: Positive with [52.3116]% accuracy



#### How This Works:
* Tokenize the new review → Converts words into numbers.
* Convert to sequences → Matches our model input format.
* Pad the sequence → Ensures consistent length (100).
* Predict sentiment → Outputs a probability between 0 and 1.
* Convert to "Positive" or "Negative" → If probability > 0.5, it's positive, else negative.